# 환경설정


In [2]:
cd /content/drive/MyDrive/kimjuyeon/NLP 스터디/nlp-code-study/02_text_relation

/content/drive/Othercomputers/내 컴퓨터/kimjuyeon/NLP 스터디/nlp-code-study/02_text_relation


# EDA(Exploratory Data Analysis)

## Import Package
* 데이터를 확인을 위한 패키지 불러오기

In [4]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

## load Data

In [ ]:
PATH = ''